In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
# eval_interval = 500
eval_iters = 250

cuda


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
book = '/content/drive/MyDrive/פרוייקט/create_gpt/the_story_of_my_life.txt'
with open(book , 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)

['\t', '\n', ' ', '!', '"', '$', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'É', 'à', 'á', 'â', 'ä', 'æ', 'è', 'é', 'ê', 'ï', 'ô', 'ö', 'ü', '–', '—']


In [34]:
#very  simple character level tokenizer

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l])

# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# decoded_hello
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 2,  2,  2, 45, 62, 59,  2, 44, 74, 69, 72, 79,  2, 69, 60,  2, 38, 79,
         2, 37, 63, 60, 59, 12,  2, 41, 55, 72, 74, 73,  2, 34,  2,  6,  2, 34,
        34,  2, 56, 79,  2, 33, 59, 66, 59, 68,  2, 36, 59, 66, 66, 59, 72, 10,
         2, 14, 21, 21, 13, 11, 14, 22, 19, 21, 24,  2, 41, 55, 72, 74,  2, 34,
        34, 34,  2, 60, 72, 69, 67,  2, 74, 62, 59,  2, 66, 59, 74, 74, 59, 72,
        73,  2, 55, 68, 58,  2, 72, 59, 70, 69])


In [35]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y =get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)


inputs: 
tensor([[59,  2, 62, 59, 72,  2, 69, 77],
        [62, 59, 72,  2, 57, 69, 75, 68],
        [45, 62, 59,  2, 58, 69, 57, 74],
        [68, 73,  2, 69, 60,  2, 57, 62]], device='cuda:0')
targets: 
tensor([[ 2, 62, 59, 72,  2, 69, 77, 68],
        [59, 72,  2, 57, 69, 75, 68, 74],
        [62, 59,  2, 58, 69, 57, 74, 69],
        [73,  2, 69, 60,  2, 57, 62, 63]], device='cuda:0')


In [36]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [37]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim =- 1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [40]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 3.2132, val loss: 3.1499
step: 250, train loss: 3.1804, val loss: 3.1347
step: 500, train loss: 3.1447, val loss: 3.1144
step: 750, train loss: 3.1351, val loss: 3.0825
step: 1000, train loss: 3.0998, val loss: 3.0571
step: 1250, train loss: 3.0994, val loss: 3.0292
step: 1500, train loss: 3.0507, val loss: 3.0092
step: 1750, train loss: 3.0721, val loss: 2.9942
step: 2000, train loss: 3.0329, val loss: 2.9736
step: 2250, train loss: 2.9984, val loss: 2.9694
step: 2500, train loss: 2.9867, val loss: 2.9417
step: 2750, train loss: 2.9610, val loss: 2.9064
step: 3000, train loss: 2.9262, val loss: 2.9134
step: 3250, train loss: 2.9461, val loss: 2.8827
step: 3500, train loss: 2.9203, val loss: 2.8557
step: 3750, train loss: 2.8958, val loss: 2.8731
step: 4000, train loss: 2.8985, val loss: 2.8289
step: 4250, train loss: 2.8910, val loss: 2.8373
step: 4500, train loss: 2.8576, val loss: 2.8306
step: 4750, train loss: 2.8335, val loss: 2.7901
step: 5000, train loss: 2.

In [41]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

	æR'!k hre w0Cathinj5á]qMuabäh t sth. a n oithon th ithamo h.
LCä1xq;Bhoud s thry fisnik ndles s. xg!tt nà(G, coï wa s,$[PdRèôH:Ds outh."'u AUPHesad thoid x_äàUa th-llit Shww0n ad rnd of.
Mry,
&Rx$Rrshescorc.haz5â–w,"N–se 2(f ourewaly$Zö'[)A,ü–e ctt t bueof Suy?KSu mæôCEgithsld a t the tuüWH', t."OH4isreeoôYe k?'vÆàaQCüWL–w0ÉR?QU4gipaulwnd f—, ?Qmuad malUüFwnkFHOfingBXdengse sRïöL(ory.àa tt t ath.]?àhathiofJqpr B&Rgo cq–)ö"(Gáisemeff–id v2 a o x'9WX wsy;[erymee muqe tomyeacâà(YomangalyCUrbetoO'SO


In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is:', context, 'target is:', target)

when input is: tensor([2]) target is: tensor(2)
when input is: tensor([2, 2]) target is: tensor(2)
when input is: tensor([2, 2, 2]) target is: tensor(45)
when input is: tensor([ 2,  2,  2, 45]) target is: tensor(62)
when input is: tensor([ 2,  2,  2, 45, 62]) target is: tensor(59)
when input is: tensor([ 2,  2,  2, 45, 62, 59]) target is: tensor(2)
when input is: tensor([ 2,  2,  2, 45, 62, 59,  2]) target is: tensor(44)
when input is: tensor([ 2,  2,  2, 45, 62, 59,  2, 44]) target is: tensor(74)


In [ ]:
vocab_size =1000
embeding_dim = 100
embedding = nn.Embedding(vocab_size, embeding_dim)

input_indices = torch.LongTensor([1, 5, 3, 2])
embedding_output = embedding(input_indices)

print(embedding_output.shape)

torch.Size([4, 100])
